In [ ]:
import psycopg2 as db
from uuid import uuid4
import hashlib
import importnb

with importnb.Notebook():
    import rag

def get_connection():
    return db.connect(
        host="localhost",
        database="metadata",
        user="postgres",
        password="khanzode"
    )

conn = get_connection()
cur = conn.cursor()

In [ ]:
def init_db():
    cur.execute("""
    CREATE TABLE IF NOT EXISTS docs (
        doc_id SERIAL PRIMARY KEY,
        doc_hash TEXT,
        doc_name TEXT UNIQUE,
        doc_chunk_ids TEXT[],
        status TEXT
    )
    """)

In [ ]:
def get_file_hash(file):
    content = file.read()
    file_hash = hashlib.sha256(content).hexdigest()
    file.seek(0)
    return file_hash
    
def is_uploaded(file):
    print("----------------------------------------------------------")
    file_hash = get_file_hash(file)
    print(f"file hash = {file_hash}")
    try:
        cur.execute("SELECT doc_hash, doc_name, doc_chunk_ids, status FROM docs WHERE doc_name = %s", (file.name,))
        returned_entry = cur.fetchone()
        print(f"returned entry = {returned_entry}")
    except:
        print("some shit happened")

    if returned_entry:
        doc_hash, doc_name, doc_chunk_ids, status = returned_entry

        if file_hash == doc_hash:   # same file same content uploaded again
            if status == "Completed":
                return 1    # 1 means that the file is the same, no need to chunk
            elif status == "Pending Vector" and (not doc_chunk_ids or len(doc_chunk_ids) == 0):
                # Allow re-chunking if stuck in Pending Vector with empty chunk_ids
                print("File stuck in Pending Vector with empty chunk_ids, allowing re-chunking.")
                return 0
            else:
                # Other non-completed statuses, allow re-chunking
                print(f"File in status {status}, allowing re-chunking.")
                return 0
        else:   # same file updated case
            cur.execute("UPDATE docs SET doc_hash = %s, doc_chunk_ids = %s, status = %s WHERE doc_name = %s", (file_hash, [], "Pending Vector", file.name))
            print("file updated")
    else:   # document inserted - add entry along with status as "Pending vector"
        cur.execute("INSERT INTO docs (doc_hash, doc_name, doc_chunk_ids, status) VALUES (%s, %s, %s, %s)",(file_hash, file.name, [], "Pending Vector"))
        print("file added for the first time")
    print("----------------------------------------------------------")

    conn.commit()
    return 0    # 0 means that the file is different or needs to be re-chunked. metadata db is updated. now get chunks and store them

def update_insert(chunk_ids, file):
    # Once chunks are completed-add chunk ids against the document
    # once entire doument is chunked and vectorisation is completed mark "completed"
    cur.execute("UPDATE docs SET doc_chunk_ids = %s, status = %s WHERE doc_name = %s",(chunk_ids, "Completed", file.name))
    print("update insert done, should show confirmed completed now")
    conn.commit()

In [ ]:
def mark_for_deletion(file):
    # Get all chunk Ids and delete those from vector
    cur.execute("SELECT chunk_ids, status from docs WHERE doc_name = %s", (file.name,))
    entry = cur.fetchone()
    chunk_ids, status = entry
    if status != "Completed":   # Do not allow update/delete while the status is not "completed"
        cur.execute("UPDATE docs SET status = %s WHERE doc_name = %s", ("Deleted Pending Vector", file.name))
        conn.commit()
        rag.delete_chunks_from_vectordb(chunk_ids)
    return chunk_ids